In [90]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist
import pandas as pd 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # suppress annoying kmeans future warning
warnings.simplefilter(action='ignore', category=DeprecationWarning)

scaler = StandardScaler()
df = pd.read_csv("data/pan22_features.csv")
X = df.drop(columns=["author_id", "discourse_type"])
cols = list(X.columns)

X_scaled = scaler.fit_transform(X)
X_T_scaled = scaler.fit_transform(X.values.T)

dis_matrix_df = pd.concat([
    pd.DataFrame(cols),
    pd.DataFrame(squareform(pdist(X_T_scaled, metric="correlation")))],
                       axis=1, ignore_index=True)

dis_matrix_df.set_index(0, inplace=True)
dis_matrix_df.columns = cols

dis_matrix_df.head()

,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,...,"('was', 'VERB')","('NOUN', 'is')","('INTJ', 'I')","('VERB', 'your')","('can', 'VERB')","('NOUN', 'was')","('NOUN', 'have')","('an', 'NOUN')","('ADV', 'the')","('n’t', 'VERB')"
0,,,,,,,,,,,,,,,,,,,,,
ADJ,0.000000,0.988814,0.771448,0.906078,1.132052,1.038133,0.835288,0.866748,1.098346,1.148486,...,1.179542,1.050419,0.975170,1.189326,1.207448,1.137160,1.199493,1.191803,1.107316,1.175719
ADP,0.988814,0.000000,1.069209,1.075143,1.029357,0.793767,1.105653,0.726901,1.046739,1.168549,...,1.108848,1.092089,1.190370,1.072902,1.057144,1.103173,1.035565,1.077695,1.089118,1.207448
ADV,0.771448,1.069209,0.000000,0.645924,1.018252,1.242100,0.627525,1.069976,1.292415,0.906844,...,1.391494,1.387928,1.134240,1.336210,1.277867,1.360370,1.479855,1.416454,1.328107,1.152493
AUX,0.906078,1.075143,0.645924,0.000000,1.254923,1.068414,0.613003,0.908586,1.389290,0.770053,...,1.406633,1.405571,1.243355,1.417378,1.285829,1.421028,1.485825,1.464667,1.449802,1.198580
CCONJ,1.132052,1.029357,1.018252,1.254923,0.000000,1.015406,1.192214,1.221713,0.940772,0.924507,...,0.713664,0.720283,0.806346,0.716796,0.682134,0.692152,0.699918,0.705725,0.691939,0.754711


In [92]:
mds_data = MDS(n_components=2, random_state=42)
mds_variables = MDS(n_components=2, random_state=42, dissimilarity="precomputed")
embedding_data = mds_data.fit_transform(X_scaled)
embedding_variables = mds_variables.fit_transform(dis_matrix_df.values, )

kmeans = KMeans(n_clusters=9, random_state=42)
kmeans.fit(X_scaled)

KMeans(n_clusters=9, random_state=42)

In [93]:

kmeans_data_df = pd.concat([pd.DataFrame({"Component 1":embedding_data[:,0],"Component 2":embedding_data[:,1]}), 
                            pd.DataFrame({"K Cluster":kmeans.labels_})], axis=1)

fig = px.scatter(kmeans_data_df, 
                 x="Component 1", 
                 y="Component 2", 
                 color="K Cluster")
fig.update_layout(title="Data MDS Plot",
                  title_x=0.5)
fig.show()




In [94]:
variables_df = pd.DataFrame({"Component 1":embedding_variables[:,0],
                             "Component 2":embedding_variables[:,1]})


fig = px.scatter(variables_df, 
                 x="Component 1", 
                 y="Component 2")
fig.update_layout(title="Variables MDS Plot",
                  title_x=0.5)

(1047,)

# K means experiment

From the plot generated below, it is difficult to see where the elbow is. Upon inspection of the points, I've determined that after `k=9` is when the points stop seeing significant change.

In [6]:
sum_squares = []
candidate_k_values = list(range(1,20))
for k in candidate_k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    sum_squares.append(kmeans.inertia_)

fig = go.Figure()
fig.add_trace(
    go.Line(
        x=candidate_k_values, 
        y=sum_squares))

fig.add_shape(
        type="line",
        x0=1, y0=max(sum_squares), x1=19, y1=min(sum_squares),
        line={"color":"grey"}
    )